import rdflib for querying the knowledge graph for action parameters

In [1]:
!pip install rdflib
# import rdflib
from rdflib import Graph, Literal, URIRef, Namespace
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc
from rdflib.namespace import OWL, RDF, RDFS

You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# We can query for the toold to be used for the chosen task:

In [3]:
g = Graph()

# define prefixes to be used in the query 
SOMA = Namespace("http://www.ease-crc.org/ont/SOMA.owl#")
CUT2 = Namespace("http://www.ease-crc.org/ont/situation_awareness#")
CUT = Namespace("http://www.ease-crc.org/ont/food_cutting#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
g.bind("owl", OWL)
g.bind("rdfs", RDFS)
g.bind("soma", SOMA)
g.bind("cut2", CUT2)
g.bind("cut", CUT)
g.bind("dul", DUL)

# the actual query comes here
query = """SELECT ?food ?alltools WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {
      ?food rdfs:subClassOf ?node.
      ?node owl:onProperty soma:hasDisposition.
      ?node owl:someValuesFrom ?collection.
      ?collection owl:intersectionOf ?node2.
      ?node2 rdf:first cut2:Cuttability.
      ?node2 rdf:rest ?toolnode.
      ?toolnode rdf:rest ?collection2.
      ?collection2 rdf:first ?tooluse.
      ?tooluse owl:onProperty soma:affordsTrigger.
      ?tooluse owl:allValuesFrom ?tool.
      ?tool owl:onProperty dul:classifies.
      ?tool owl:allValuesFrom ?whattool.
      ?alltools rdfs:subClassOf* ?whattool.} }"""

for row in g.query(query):
    print(row.alltools)

# The robot needs to know if prior actions need to be executed:

In [14]:
additionalactionquery = """SELECT ?food ?action WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  ?food rdfs:subClassOf ?restriction.
  ?restriction owl:onProperty cut:hasPart.
  ?restriction owl:someValuesFrom ?node.
  ?node owl:intersectionOf ?intersec.
  ?intersec rdf:first ?foodpart.
  ?intersec rdf:rest ?node2.
  ?node2 rdf:first ?part.
  ?part owl:onProperty cut:hasEdibility.
  {?part owl:someValuesFrom cut:ShouldBeAvoided.}
  UNION
  {?part owl:someValuesFrom cut:MustBeAvoided.}
  ?foodpart rdfs:subClassOf ?partrestriction.
  ?partrestriction owl:onProperty soma:hasDisposition.
  ?partrestriction owl:someValuesFrom ?partnode.
  ?partnode owl:intersectionOf ?intersection.
  ?intersection rdf:first ?disposition.
  ?intersection rdf:rest ?actionrestriction.
  ?actionrestriction rdf:first ?actionnode.
  ?actionnode owl:onProperty soma:affordsTask.
  ?actionnode owl:someValuesFrom ?action.
} }"""

for row in g.query(additionalactionquery):
    print(row.food, row.action)

http://purl.obolibrary.org/obo/FOODON_03309927 http://www.ease-crc.org/ont/food_cutting#StemRemoving
http://purl.obolibrary.org/obo/FOODON_00003520 http://www.ease-crc.org/ont/food_cutting#StemRemoving
http://purl.obolibrary.org/obo/FOODON_00003415 http://www.ease-crc.org/ont/food_cutting#StemRemoving
http://purl.obolibrary.org/obo/FOODON_00003486 http://www.ease-crc.org/ont/food_cutting#Peeling
http://purl.obolibrary.org/obo/FOODON_00003539 http://www.ease-crc.org/ont/food_cutting#Peeling
http://purl.obolibrary.org/obo/FOODON_00003443 http://www.ease-crc.org/ont/food_cutting#StemRemoving
http://purl.obolibrary.org/obo/FOODON_03301710 http://www.ease-crc.org/ont/food_cutting#CoreCutting
http://purl.obolibrary.org/obo/FOODON_03301710 http://www.ease-crc.org/ont/food_cutting#CoreScraping
http://purl.obolibrary.org/obo/FOODON_03309832 http://www.ease-crc.org/ont/food_cutting#Peeling
http://purl.obolibrary.org/obo/FOODON_00003459 http://www.ease-crc.org/ont/food_cutting#Peeling
http://purl

In [15]:
prioractionquery = """  SELECT ?task ?priortask WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  ?task rdfs:subClassOf ?sub.
  ?sub owl:onProperty cut:requiresPriorTask .
  ?sub owl:someValuesFrom ?priortask.
} }"""

for row in g.query(prioractionquery):
    print(row.task, row.priortask)

http://www.ease-crc.org/ont/food_cutting#Julienning http://www.ease-crc.org/ont/SOMA.owl#Slicing
http://www.ease-crc.org/ont/food_cutting#Quartering http://www.ease-crc.org/ont/food_cutting#Halving
http://www.ease-crc.org/ont/SOMA.owl#Dicing http://www.ease-crc.org/ont/food_cutting#Julienning


# The robot needs to also know the number of repetitions required for performing the task:

In [4]:
repetitionsquery = """  SELECT ?task ?res WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  {
      OPTIONAL{?task rdfs:subClassOf ?action}
        ?action rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {
            ?rep_node owl:hasValue ?val.
            }
        BIND("1" AS ?res)
    }
    UNION
    {
        OPTIONAL{?task rdfs:subClassOf ?action}
        ?action rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {
            ?rep_node owl:minQualifiedCardinality ?val.
        }
        BIND("more than 1" AS ?res)
    }
} }"""

for row in g.query(repetitionsquery):
    print(row.task, row.res)

# Last thin to know before performing the task: The position needed for action execution:

In [21]:
positionquery = """  SELECT ?task ?position WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  OPTIONAL {?task rdfs:subClassOf ?sub.}
  ?sub rdfs:subClassOf* ?node.
  ?node owl:onProperty cut:affordsPosition.
  ?node owl:someValuesFrom ?position.
} }"""

for row in g.query(positionquery):
    print(row.task, row.position)

http://www.ease-crc.org/ont/food_cutting#Halving http://www.ease-crc.org/ont/food_cutting#halving_position
http://www.ease-crc.org/ont/SOMA.owl#Dicing http://www.ease-crc.org/ont/food_cutting#slicing_position
http://www.ease-crc.org/ont/food_cutting#Carving http://www.ease-crc.org/ont/food_cutting#slicing_position
http://www.ease-crc.org/ont/food_cutting#Paring http://www.ease-crc.org/ont/food_cutting#slicing_position
http://www.ease-crc.org/ont/food_cutting#Sawing http://www.ease-crc.org/ont/food_cutting#slicing_position
http://www.ease-crc.org/ont/food_cutting#Quartering http://www.ease-crc.org/ont/food_cutting#halving_position
http://www.ease-crc.org/ont/food_cutting#Julienning http://www.ease-crc.org/ont/food_cutting#slicing_position
http://www.ease-crc.org/ont/SOMA.owl#Slicing http://www.ease-crc.org/ont/food_cutting#slicing_position
http://www.ease-crc.org/ont/SOMA.owl#Cutting http://www.ease-crc.org/ont/food_cutting#slicing_position
http://www.ease-crc.org/ont/food_cutting#Chopp